In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import calendar

import folium

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [2]:
color_dark = '#000000'
color_light = '#FFFFFF'
color_dark_accent = '#14213D'
color_light_accent = '#E5E5E5'
color_highlight = '#FCA311'
color_dark_accent_2 = '#3d3d3d'

In [3]:
def convert_timestring_to_time(date_string):
    if date_string is np.nan:
        return date_string
    idx = date_string.find('M')
    if (idx == -1):
        return datetime.datetime.strptime(date_string, '%H:%M')
    idx -= 2
    if date_string[idx] == ' ':
        return datetime.datetime.strptime(date_string, '%I:%M %p')
    return datetime.datetime.strptime(date_string, '%I:%M%p')

In [4]:
df = pd.read_csv("../../data/school_shooting_data.csv")
df['time'] = df['time'].apply(lambda x: convert_timestring_to_time(x))
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

In [5]:
df = df[df['long'].notna()]
df.reset_index(drop=True, inplace=True)

In [6]:
shootings_per_year = df.groupby(['year']).agg(
    sum_killed=('killed', 'sum'),
    sum_injured=('injured', 'sum'),
    sum=('year', 'count')
).reset_index()
shootings_per_year.head()

,year,sum_killed,sum_injured,sum
0,1999,14,35,7
1,2000,4,3,12
2,2001,5,22,13
3,2002,1,7,5
4,2003,5,8,12


In [7]:
labels = {"sum_killed": "Killed", "sum_injured": "Injured"}
fig = px.bar(shootings_per_year, x="year", y=[
             "sum_killed", "sum_injured"], height=400)
fig.for_each_trace(lambda t: t.update(name=labels[t.name],
                                      legendgroup=labels[t.name],
                                      hovertemplate=t.hovertemplate.replace(
                                          t.name, labels[t.name])
                                      )
                   )
fig.update_traces(hovertemplate='Year: %{x} <br> Victimes: %{y}')
fig.update_layout(legend_title="Victims",
                  xaxis_title="Year", yaxis_title="Count")
fig.show()


In [8]:
fig = go.Figure(
    data=[
        go.Bar(
            x=shootings_per_year['year'],
            y=shootings_per_year['sum_injured'],
            offsetgroup=0, name='People injured',
            hovertemplate='Year: %{x}<br>Injured: %{y}',
            hovertextsrc='',
        ),
        go.Bar(
            x=shootings_per_year['year'],
            y=shootings_per_year['sum_killed'],
            offsetgroup=0,
            base=shootings_per_year['sum_injured'],
            name='People killed',
            hovertemplate='Year: %{x}<br>Killed: %{y}'
        ),
        go.Line(
            x=shootings_per_year['year'],
            y=shootings_per_year['sum'],
            name='Shootings count',
            hovertemplate='Year: %{x}<br>Shootings occured: %{y}',
        ),
    ],
)

fig.update_layout(
    width=630, 
    height=500,
    title="School shooting casualties vs occurences per year",
    title_x=.5,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=1,
        showline=True,
        mirror=True,
        linecolor="#000000"
    ),
    yaxis=dict(
        showline=True,
        mirror=True,
        linecolor="#000000",
    ),
    xaxis_title="Year",
    yaxis_title="Count of casualties/occurences",
    font=dict(family='Rubik', size=12, color=color_dark),
    plot_bgcolor=color_light,
    modebar=dict(
        orientation='v'
    ),
    colorway=[color_dark_accent_2, color_dark_accent, color_highlight]
)
fig.show()


c:\Users\j\miniconda3\envs\social-data-viz\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [9]:
chart_div = plotly.offline.plot(fig, output_type='div')
with open("../docs/_includes/shootings_details.html", 'w') as f:
    f.write(chart_div)